# Linear Learner with HPO

In [9]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri 
from sagemaker.session import s3_input, Session

ll_container =  get_image_uri(boto3.Session().region_name,
                          'linear-learner')

# initialize hyperparameters
hyperparameters = {
        "predictor_type":"multiclass_classifier",
        "num_classes": "3"     }

bucket = "preshen-sagemaker"
prefix = "tuning-lab"

output_path = "s3://{}/{}/{}/".format(bucket, prefix, 'output')
ll_estimator = sagemaker.estimator.Estimator(image_name=ll_container, 
                                          role=sagemaker.get_execution_role(),
                                          hyperparameters=hyperparameters,
                                          train_instance_count=1, 
                                          train_instance_type='ml.m5.xlarge', 
                                          train_volume_size=5, # 5 GB 
                                          output_path=output_path)

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


In [17]:
# define the data type and paths to the training and validation datasets

prefix = "algorithm-lab"


content_type = "text/csv"
train_input = s3_input("s3://{}/{}/{}/".format(bucket, prefix, 'train'), content_type=content_type)
validation_input = s3_input("s3://{}/{}/{}/".format(bucket, prefix, 'validation'), content_type=content_type)


's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


In [18]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner


hyperparameter_ranges_linear = {
    'l1': ContinuousParameter(0.000001, 1, scaling_type="Linear"),
    'learning_rate': ContinuousParameter(0.00001, 1, scaling_type="Linear"),
    'mini_batch_size': IntegerParameter(100,5000),
    'use_bias': CategoricalParameter([True, False]),
    'positive_example_weight_mult': ContinuousParameter(0.0001, 10)
}


In [19]:
objective_metric_name= 'validation:multiclass_accuracy'

In [20]:
tuner_linear = HyperparameterTuner(
    ll_estimator,
    objective_metric_name,
    hyperparameter_ranges_linear,
    max_jobs=50,
    max_parallel_jobs=10,
    strategy='Bayesian'
)

In [21]:
tuner_linear.fit({'train': train_input, 'validation': validation_input}, include_cls_metadata=False)